In [2]:
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
from unidecode import unidecode  # Remove acentos

In [3]:
# ========================== Funções Auxiliares =================================================================================
def fazer_requisicao(url):
    """Faz a requisição à API e retorna os dados em JSON"""
    try:
        resposta = requests.get(url)
        resposta.raise_for_status()  # Levanta um erro para status 4xx e 5xx
        return resposta.json().get('dados', [])
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return []

In [4]:
# ========================== Funções Auxiliares =================================================================================
def obter_deputadas():
    """Obtém a lista de deputadas na Câmara"""
    start_time = time.time()  # Início da contagem do tempo
    url = 'https://dadosabertos.camara.leg.br/api/v2/deputados?dataInicio=2020-01-01&dataFim=2024-12-31&ordem=ASC&ordenarPor=nome'
    dados = fazer_requisicao(url)
    end_time = time.time()  # Fim da contagem do tempo
    print(f"Tempo total de execução: {end_time - start_time:.2f} segundos")
    return pd.DataFrame(dados) if dados else pd.DataFrame()

In [5]:
# ========================== Extração de dados =================================================================================
print("Baixando dados das deputadas...")
df_deputadas = obter_deputadas()

Baixando dados das deputadas...
Tempo total de execução: 2.91 segundos


In [6]:
# ========================== Funções Auxiliares =================================================================================
def obter_detalhes_deputadas(df_deputadas):
    """Obtém detalhes individuais das deputadas e retorna um DataFrame"""
    start_time = time.time()
    detalhes = []

    for id_dep in df_deputadas['id']:
        url = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{id_dep}"
        data = fazer_requisicao(url)  # Sua função para obter os dados da API

        if not data:
            print(f"[ERRO] Nenhum dado retornado para id {id_dep}")
            continue

        # Converter os dados principais da deputada para DataFrame
        df_dados_deputada = pd.json_normalize(data)

        detalhes.append(df_dados_deputada)

    # Consolidar todos os detalhes em um único DataFrame
    df_resultado = pd.concat(detalhes, ignore_index=True) if detalhes else pd.DataFrame()

    elapsed_time = time.time() - start_time
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")

    return df_resultado

In [7]:
# ========================== Extração de dados =================================================================================
if not df_deputadas.empty:
    print("Baixando detalhes das deputadas...")
    df_deputadas_detalhes = obter_detalhes_deputadas(df_deputadas)
else:
    df_deputadas_detalhes = pd.DataFrame()

Baixando detalhes das deputadas...
Tempo total de execução: 171.40 segundos


In [8]:
# Junção das tabelas deputadas
df_deputadas_completo = df_deputadas.merge(df_deputadas_detalhes, on="id", how="left")

In [9]:
def obter_partidos(df_deputadas_completo):
    """Obtém detalhes dos partidos a partir da sigla e retorna um DataFrame"""
    start_time = time.time()  # Início da contagem do tempo
    url_base = "https://dadosabertos.camara.leg.br/api/v2/partidos?sigla={}&ordem=ASC&ordenarPor=sigla"
    siglas = df_deputadas_completo["siglaPartido"].dropna().unique()  # Pega siglas únicas, ignorando NaN
    dados_partidos = []

    for sigla in siglas:
        sigla_corrigida = unidecode(sigla).upper()  # Remove acentos e coloca em maiúsculas
        url = url_base.format(sigla_corrigida)  
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            if "dados" in data and len(data["dados"]) > 0:
                df_partido = pd.json_normalize(data["dados"])
                dados_partidos.append(df_partido)

    # Junta todos os dados coletados em um único DataFrame
    df_partidos = pd.concat(dados_partidos, ignore_index=True) if dados_partidos else pd.DataFrame()

    end_time = time.time()  # Fim da contagem do tempo
    print(f"Tempo total de execução: {end_time - start_time:.2f} segundos")

    return df_partidos

In [10]:
print("Baixando dados dos partidos...")
df_partidos = obter_partidos(df_deputadas_completo)

Baixando dados dos partidos...
Tempo total de execução: 43.97 segundos


In [11]:
def obter_detalhes_partidos(df_partidos):
    """
    Obtém os detalhes de cada partido utilizando o ID da lista de partidos.

    Parâmetros:
        df_partidos (pd.DataFrame): DataFrame contendo a lista de partidos, incluindo a coluna 'id'.

    Retorna:
        pd.DataFrame: DataFrame com os detalhes dos partidos, incluindo campos expandidos do 'status'.
    """
    start_time = time.time()  # Início da contagem do tempo
    detalhes_partidos = []  # Lista para armazenar os detalhes

    for partido_id in df_partidos["id"]:
        url = f"https://dadosabertos.camara.leg.br/api/v2/partidos/{partido_id}"
        dados = fazer_requisicao(url)

        if dados:
            # Converte os dados principais do partido para um DataFrame
            df_partido = pd.json_normalize(dados)

            # Expande os campos dentro de "status", se existir
            if "status" in dados:
                df_status = pd.json_normalize(dados["status"])
                df_detalhado = pd.concat([df_partido, df_status], axis=1)  # Junta os DataFrames
            else:
                df_detalhado = df_partido  # Caso não haja status, mantém apenas os dados básicos

            detalhes_partidos.append(df_detalhado)

    end_time = time.time()  # Fim da contagem do tempo
    print(f"Tempo total de execução: {end_time - start_time:.2f} segundos")

    # Concatena todos os DataFrames na lista em um único DataFrame
    return pd.concat(detalhes_partidos, ignore_index=True) if detalhes_partidos else pd.DataFrame()

print("Baixando detalhes dos partidos...")
df_detalhes_partidos = obter_detalhes_partidos(df_partidos)

Baixando detalhes dos partidos...
Tempo total de execução: 5.25 segundos


In [12]:
# Junção das tabelas partidos
df_partidos_completo = df_partidos.merge(df_detalhes_partidos, on="id", how="left")

In [13]:
def determinar_geracao(ano_nascimento):
    if ano_nascimento >= 1997:
        return "Geração Z (1997-2010)"
    elif 1981 <= ano_nascimento <= 1996:
        return "Millennials - Geração Y (1981-1996)"
    elif 1965 <= ano_nascimento <= 1980:
        return "Geração X (1965-1980)"
    elif 1946 <= ano_nascimento <= 1964:
        return "Baby Boomers (1946-1964)"
    else:
        return "Geração Silenciosa (até 1945)"

In [14]:
# ========================== Consolidação e Limpeza ==========================

In [15]:
df_deputadas_completo.isnull().sum()

id                                   0
uri_x                                0
nome                                 0
siglaPartido                         0
uriPartido                           0
siglaUf                              0
idLegislatura                        0
urlFoto                              0
email                             1598
uri_y                                0
nomeCivil                            0
cpf                                  0
sexo                                 0
urlWebsite                        2262
redeSocial                           0
dataNascimento                       0
dataFalecimento                   2274
ufNascimento                         1
municipioNascimento                  0
escolaridade                        33
ultimoStatus.id                      0
ultimoStatus.uri                     0
ultimoStatus.nome                    0
ultimoStatus.siglaPartido            0
ultimoStatus.uriPartido           2278
ultimoStatus.siglaUf     

In [16]:
df_deputadas_completo.loc[(df_deputadas_completo['ufNascimento'].isna()), 'ufNascimento'] = df_deputadas_completo['siglaUf']

In [17]:
df_deputadas_completo.loc[(df_deputadas_completo['escolaridade'].isna()), 'escolaridade'] = "Não informado"

In [18]:
df_deputadas_completo.groupby('idLegislatura')['idLegislatura'].nunique(dropna=False)

idLegislatura
56    1
57    1
Name: idLegislatura, dtype: int64

In [19]:
df_deputadas_completo.groupby('siglaPartido')['siglaPartido'].count()

siglaPartido
AVANTE            21
CIDADANIA         25
DEM               64
MDB              119
NOVO              14
PATRIOTA          19
PCdoB             13
PDT               58
PHS                4
PL               369
PODE              45
PP               176
PPS                2
PR                13
PRB                5
PRD               12
PROS              26
PSB               59
PSC               55
PSD              157
PSDB              72
PSL              165
PSOL              26
PT               132
PTB               16
PV                26
REDE               1
REPUBLICANOS     165
S.PART.           20
SOLIDARIEDADE     26
UNIÃO            373
Name: siglaPartido, dtype: int64

In [20]:
df_deputadas_completo['siglaPartido'].nunique()

31

In [21]:
df_deputadas_completo["inicioLegislatura"] = np.where(
    df_deputadas_completo["idLegislatura"] == 57, "2023-02-01",
    np.where(df_deputadas_completo["idLegislatura"] == 56, "2019-02-01", np.nan)
)

df_deputadas_completo["fimLegislatura"] = np.where(
    df_deputadas_completo["idLegislatura"] == 57, "2027-01-31",
    np.where(df_deputadas_completo["idLegislatura"] == 56, "2023-01-31", np.nan)
)

In [22]:
df_deputadas_completo.duplicated(subset=['id', 'idLegislatura', 'siglaPartido']).sum()

1294

In [23]:
df_deputadas_completo = df_deputadas_completo.drop_duplicates(subset=["id", "idLegislatura", 'siglaPartido'], keep='last')

In [24]:
df_deputadas_completo.duplicated(subset=['id', 'idLegislatura', 'siglaPartido']).sum()

0

In [25]:
df_deputadas_completo['siglaPartido'].nunique()

31

In [26]:
df_deputadas_completo.groupby('sexo', dropna=False)['id'].nunique(dropna=False)

sexo
F    103
M    449
Name: id, dtype: int64

In [27]:
df_deputadas_completo['dataNascimento'] = pd.to_datetime(df_deputadas_completo['dataNascimento']).dt.strftime('%d/%m/%Y')

In [28]:
df_deputadas_completo['inicioLegislatura'] = pd.to_datetime(df_deputadas_completo['inicioLegislatura']).dt.strftime('%d/%m/%Y')

In [29]:
df_deputadas_completo['fimLegislatura'] = pd.to_datetime(df_deputadas_completo['fimLegislatura']).dt.strftime('%d/%m/%Y')

In [30]:
df_deputadas_completo['falecida'] = df_deputadas_completo['dataFalecimento'].notna()

In [31]:
df_deputadas_completo.isnull().sum()

id                                  0
uri_x                               0
nome                                0
siglaPartido                        0
uriPartido                          0
siglaUf                             0
idLegislatura                       0
urlFoto                             0
email                             645
uri_y                               0
nomeCivil                           0
cpf                                 0
sexo                                0
urlWebsite                        976
redeSocial                          0
dataNascimento                      0
dataFalecimento                   980
ufNascimento                        0
municipioNascimento                 0
escolaridade                        0
ultimoStatus.id                     0
ultimoStatus.uri                    0
ultimoStatus.nome                   0
ultimoStatus.siglaPartido           0
ultimoStatus.uriPartido           984
ultimoStatus.siglaUf                0
ultimoStatus

In [32]:
df_deputadas_completo.loc[:, 'ano_nascimento'] = pd.to_datetime(df_deputadas_completo['dataNascimento'], dayfirst=True).dt.year

In [33]:
df_deputadas_completo.loc[:, 'geração'] = df_deputadas_completo['ano_nascimento'].apply(determinar_geracao)

In [34]:
df_deputadas_completo['geração'].value_counts()

geração
Geração X (1965-1980)                  410
Baby Boomers (1946-1964)               319
Millennials - Geração Y (1981-1996)    234
Geração Silenciosa (até 1945)           16
Geração Z (1997-2010)                    5
Name: count, dtype: int64

In [35]:
df_deputadas_completo.loc[:, 'idade'] = datetime.now().year - df_deputadas_completo['ano_nascimento']

In [36]:
df_deputadas_completo.drop(columns=['uri_x', 'uriPartido', 'urlFoto', 'email', 'uri_y', 'urlWebsite', 'redeSocial', 
                                    'ultimoStatus.uriPartido', 'ultimoStatus.email', 'ultimoStatus.gabinete.nome', 
                                    'ultimoStatus.gabinete.predio', 'ultimoStatus.gabinete.sala', 'ultimoStatus.gabinete.andar',
                                    'ultimoStatus.gabinete.telefone', 'ultimoStatus.gabinete.email', 'ultimoStatus.descricaoStatus', 
                                    'dataFalecimento'], inplace=True)

In [37]:
df_deputadas_completo.head()

,id,nome,siglaPartido,siglaUf,idLegislatura,nomeCivil,cpf,sexo,dataNascimento,ufNascimento,...,ultimoStatus.data,ultimoStatus.nomeEleitoral,ultimoStatus.situacao,ultimoStatus.condicaoEleitoral,inicioLegislatura,fimLegislatura,falecida,ano_nascimento,geração,idade
0,220593,Abilio Brunini,PL,MT,57,ABILIO JACQUES BRUNINI MOUMER,99770962104,M,31/01/1984,MT,...,2025-01-01,Abilio Brunini,Vacância,Titular,01/02/2023,31/01/2027,False,1984,Millennials - Geração Y (1981-1996),41
2,204554,Abílio Santana,PL,BA,56,JOSE ABILIO SILVA DE SANTANA,36607606504,M,13/02/1965,BA,...,2023-01-31,Abílio Santana,Fim de Mandato,Titular,01/02/2019,31/01/2023,False,1965,Geração X (1965-1980),60
4,204554,Abílio Santana,PSC,BA,56,JOSE ABILIO SILVA DE SANTANA,36607606504,M,13/02/1965,BA,...,2023-01-31,Abílio Santana,Fim de Mandato,Titular,01/02/2019,31/01/2023,False,1965,Geração X (1965-1980),60
6,204521,Abou Anni,PSL,SP,56,PAULO SERGIO ABOU ANNI,08496582841,M,06/11/1966,SP,...,2023-01-31,Abou Anni,Fim de Mandato,Titular,01/02/2019,31/01/2023,False,1966,Geração X (1965-1980),59
8,204521,Abou Anni,UNIÃO,SP,56,PAULO SERGIO ABOU ANNI,08496582841,M,06/11/1966,SP,...,2023-01-31,Abou Anni,Fim de Mandato,Titular,01/02/2019,31/01/2023,False,1966,Geração X (1965-1980),59


In [38]:
# ========================== Consolidação e Limpeza ==========================

In [39]:
df_partidos_completo.isnull().sum()

id                             0
sigla_x                        0
nome_x                         0
uri_x                          0
sigla_y                        0
nome_y                         0
uri_y                          0
numeroEleitoral               31
urlLogo                        0
urlWebSite                    31
urlFacebook                   31
status.data                    1
status.idLegislatura           0
status.situacao                1
status.totalPosse              0
status.totalMembros            0
status.uriMembros              0
status.lider.uri               0
status.lider.nome              1
status.lider.siglaPartido      1
status.lider.uriPartido        0
status.lider.uf                1
status.lider.idLegislatura     0
status.lider.urlFoto           0
data                           1
idLegislatura                  0
situacao                       1
totalPosse                     0
totalMembros                   0
uriMembros                     0
lider.uri 

In [40]:
df_partidos_completo = df_partidos_completo.rename(columns={'id': 'idPartido', 'sigla_x': 'siglaPartido', 'nome_x': 'nomePartido'})

In [41]:
df_partidos_completo.drop(columns=['uri_x', 'uri_y', 'urlLogo', 'urlWebSite', 'urlFacebook', 'status.lider.uri', 
                                   'status.lider.uriPartido', 'status.lider.urlFoto', 'uriMembros', 'lider.uri', 
                                   'lider.uriPartido', 'lider.urlFoto', 'uriMembros', 'lider.uri', 'lider.uriPartido', 
                                   'lider.urlFoto', 'sigla_y', 'nome_y', 'numeroEleitoral'], inplace=True)

In [42]:
df_partidos_completo['status.data'] = pd.to_datetime(df_partidos_completo['status.data']).dt.strftime("%d/%m/%Y")

In [43]:
df_partidos_completo['data'] = pd.to_datetime(df_partidos_completo['data']).dt.strftime("%d/%m/%Y")

In [44]:
df_partidos_completo.isnull().sum()

idPartido                     0
siglaPartido                  0
nomePartido                   0
status.data                   1
status.idLegislatura          0
status.situacao               1
status.totalPosse             0
status.totalMembros           0
status.uriMembros             0
status.lider.nome             1
status.lider.siglaPartido     1
status.lider.uf               1
status.lider.idLegislatura    0
data                          1
idLegislatura                 0
situacao                      1
totalPosse                    0
totalMembros                  0
lider.nome                    1
lider.siglaPartido            1
lider.uf                      1
lider.idLegislatura           0
dtype: int64

In [45]:
df_partidos_completo.head()

,idPartido,siglaPartido,nomePartido,status.data,status.idLegislatura,status.situacao,status.totalPosse,status.totalMembros,status.uriMembros,status.lider.nome,...,status.lider.idLegislatura,data,idLegislatura,situacao,totalPosse,totalMembros,lider.nome,lider.siglaPartido,lider.uf,lider.idLegislatura
0,37906,PL,Partido Liberal,28/02/2025,57,Ativo,99,92,https://dadosabertos.camara.leg.br/api/v2/depu...,Giacobo,...,57,28/02/2025,57,Ativo,99,92,Giacobo,PL,PR,57
1,36833,PSC,Partido Social Cristão,16/02/2022,57,Extinto,0,0,https://dadosabertos.camara.leg.br/api/v2/depu...,Euclydes Pettersen,...,57,16/02/2022,57,Extinto,0,0,Euclydes Pettersen,PSC,MG,57
2,36837,PSL,Partido Social Liberal,13/12/2018,56,Inativo,0,0,https://dadosabertos.camara.leg.br/api/v2/depu...,DELEGADO WALDIR,...,56,13/12/2018,56,Inativo,0,0,DELEGADO WALDIR,PSL,GO,56
3,38009,UNIÃO,União Brasil,01/02/2025,57,Ativo,59,59,https://dadosabertos.camara.leg.br/api/v2/depu...,Pedro Lucas Fernandes,...,57,01/02/2025,57,Ativo,59,59,Pedro Lucas Fernandes,UNIÃO,MA,57
4,36763,PROS,Partido Republicano da Ordem Social,12/08/2022,57,Extinto,0,0,https://dadosabertos.camara.leg.br/api/v2/depu...,Weliton Prado,...,57,12/08/2022,57,Extinto,0,0,Weliton Prado,PROS,MG,57


df_diff = df_partidos_completo[df_partidos_completo['siglaPartido'] != df_partidos_completo['sigla_y']]
print(df_diff)

In [46]:
print(df_deputadas_completo['siglaPartido'].isnull().sum())
print(df_partidos_completo['siglaPartido'].isnull().sum())

0
0


In [47]:
df_partidos_completo[df_partidos_completo.isnull().any(axis=1)]

,idPartido,siglaPartido,nomePartido,status.data,status.idLegislatura,status.situacao,status.totalPosse,status.totalMembros,status.uriMembros,status.lider.nome,...,status.lider.idLegislatura,data,idLegislatura,situacao,totalPosse,totalMembros,lider.nome,lider.siglaPartido,lider.uf,lider.idLegislatura
25,36852,S.PART.,Sem Partido,NaN,57,None,0,1,,None,...,57,NaN,57,None,0,1,None,None,None,57


In [48]:
df_partidos_completo.isnull().sum()[df_partidos_completo.isnull().sum() > 0]

status.data                  1
status.situacao              1
status.lider.nome            1
status.lider.siglaPartido    1
status.lider.uf              1
data                         1
situacao                     1
lider.nome                   1
lider.siglaPartido           1
lider.uf                     1
dtype: int64

In [49]:
df_partidos_completo.fillna({
    'status.data': 'Não se aplica',
    'status.situacao': 'Não se aplica',
    'status.lider.nome': 'Não se aplica',
    'status.lider.siglaPartido': df_partidos_completo['siglaPartido'],
    'status.lider.uf': 'Não se aplica',
    'data': 'Não se aplica',
    'situacao': 'Não se aplica',
    'lider.nome': 'Não se aplica',
    'lider.siglaPartido': df_partidos_completo['siglaPartido'],
    'lider.uf': 'Não se aplica' 
}, inplace=True)

In [50]:
df_partidos_completo.isnull().sum()

idPartido                     0
siglaPartido                  0
nomePartido                   0
status.data                   0
status.idLegislatura          0
status.situacao               0
status.totalPosse             0
status.totalMembros           0
status.uriMembros             0
status.lider.nome             0
status.lider.siglaPartido     0
status.lider.uf               0
status.lider.idLegislatura    0
data                          0
idLegislatura                 0
situacao                      0
totalPosse                    0
totalMembros                  0
lider.nome                    0
lider.siglaPartido            0
lider.uf                      0
lider.idLegislatura           0
dtype: int64

In [51]:
df_partidos_completo.info

<bound method DataFrame.info of     idPartido   siglaPartido                              nomePartido  \
0       37906             PL                          Partido Liberal   
1       36833            PSC                   Partido Social Cristão   
2       36837            PSL                   Partido Social Liberal   
3       38009          UNIÃO                             União Brasil   
4       36763           PROS      Partido Republicano da Ordem Social   
5       36899            MDB         Movimento Democrático Brasileiro   
6       37908   REPUBLICANOS                             Republicanos   
7       36835           PSDB  Partido da Social Democracia Brasileira   
8       37901           NOVO                             Partido Novo   
9       37903             PP                            Progressistas   
10      36814             PR                     Partido da República   
11      36844             PT                Partido dos Trabalhadores   
12      36786      

In [52]:
df_deputadas_partidos = df_deputadas_completo.merge(df_partidos_completo, on='siglaPartido', how='left')

In [53]:
df_deputadas_partidos.head()

,id,nome,siglaPartido,siglaUf,idLegislatura_x,nomeCivil,cpf,sexo,dataNascimento,ufNascimento,...,status.lider.idLegislatura,data,idLegislatura_y,situacao,totalPosse,totalMembros,lider.nome,lider.siglaPartido,lider.uf,lider.idLegislatura
0,220593,Abilio Brunini,PL,MT,57,ABILIO JACQUES BRUNINI MOUMER,99770962104,M,31/01/1984,MT,...,57,28/02/2025,57,Ativo,99,92,Giacobo,PL,PR,57
1,204554,Abílio Santana,PL,BA,56,JOSE ABILIO SILVA DE SANTANA,36607606504,M,13/02/1965,BA,...,57,28/02/2025,57,Ativo,99,92,Giacobo,PL,PR,57
2,204554,Abílio Santana,PSC,BA,56,JOSE ABILIO SILVA DE SANTANA,36607606504,M,13/02/1965,BA,...,57,16/02/2022,57,Extinto,0,0,Euclydes Pettersen,PSC,MG,57
3,204521,Abou Anni,PSL,SP,56,PAULO SERGIO ABOU ANNI,08496582841,M,06/11/1966,SP,...,56,13/12/2018,56,Inativo,0,0,DELEGADO WALDIR,PSL,GO,56
4,204521,Abou Anni,UNIÃO,SP,56,PAULO SERGIO ABOU ANNI,08496582841,M,06/11/1966,SP,...,57,01/02/2025,57,Ativo,59,59,Pedro Lucas Fernandes,UNIÃO,MA,57


In [54]:
df_deputadas_partidos.loc[(df_deputadas_partidos['ultimoStatus.data'].isna()), 'ultimoStatus.data'] = df_deputadas_partidos['status.data']

In [55]:
df_deputadas_partidos.isnull().sum()

id                                0
nome                              0
siglaPartido                      0
siglaUf                           0
idLegislatura_x                   0
nomeCivil                         0
cpf                               0
sexo                              0
dataNascimento                    0
ufNascimento                      0
municipioNascimento               0
escolaridade                      0
ultimoStatus.id                   0
ultimoStatus.uri                  0
ultimoStatus.nome                 0
ultimoStatus.siglaPartido         0
ultimoStatus.siglaUf              0
ultimoStatus.idLegislatura        0
ultimoStatus.urlFoto              0
ultimoStatus.data                 0
ultimoStatus.nomeEleitoral        0
ultimoStatus.situacao             0
ultimoStatus.condicaoEleitoral    0
inicioLegislatura                 0
fimLegislatura                    0
falecida                          0
ano_nascimento                    0
geração                     

df_deputadas_completo.to_csv("deputadas.csv", index=False)  # Sem o índice
df_partidos_completo.to_csv("partidos.csv", index=False)  # Sem o índice

In [75]:
dados = fazer_requisicao('https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=56&siglaSexo=F&ordem=ASC&ordenarPor=nome')
df_deputadas_mulheres_56 = pd.DataFrame(dados)

dados = fazer_requisicao('https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=57&siglaSexo=F&ordem=ASC&ordenarPor=nome')
df_deputadas_mulheres_57 = pd.DataFrame(dados)

df_deputadas_mulheres = pd.concat([df_deputadas_mulheres_56, df_deputadas_mulheres_57], verify_integrity=True, ignore_index=True)
df_deputadas_mulheres.to_csv('deputadas_mulheres.csv', index=False)

dados = fazer_requisicao('https://dadosabertos.camara.leg.br/api/v2/deputados?siglaSexo=M&ordem=ASC&ordenarPor=nome')
df_deputados_homens = pd.DataFrame(dados)
df_deputados_homens.to_csv('deputados_homens.csv', index=False)

dados = fazer_requisicao('https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=56&siglaSexo=M&ordem=ASC&ordenarPor=nome')
df_deputados_homens_56 = pd.DataFrame(dados)
df_deputados_homens_56.to_csv('deputados_homens_56.csv', index=False)